# Optional: Setup an external Model Serving Endpoint

The Model Serving Endpoints used by your AI function can be of any type.

You can setup an endpoint to hit an external Foundation Model, such as Azure OpenAI.

This let you define your secret and connection setup once, and allow other users to access your external model, ensuring security, cost control and traceability.

These can be of different flavor: Azure OpenAI, Anthropic, OpenAI...

## Setting up Azure OpenAI

To be able to call our AI function, we'll need a key to query the Azure OpenAI API. Here are the steps to retrieve your key:

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-function-setup-1.png" width="1250px">

- Navigate to [Azure Portal > All Services > Cognitive Services > Azure OpenAI](https://portal.azure.com/#view/Microsoft_Azure_ProjectOxford/CognitiveServicesHub/~/OpenAI)  
- Click into the relevant resource
- Click on `Keys and Endpoint`
  - Copy down your:
    - **Key** (do **NOT** store this in plaintext, include it in your code, and/or commit it into your git repo)
    - **Resource name** which is represented in the endpoint URL (of the form `https://<resource-name>.openai.azure.com/`)
- Click on `Model deployments`
  - Note down your **model deployment name**

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=DBSQL&org_id=1444828305810485&notebook=%2F05-Extra-setup-external-model-OpenAI&demo_name=sql-ai-functions&event=VIEW&path=%2F_dbdemos%2FDBSQL%2Fsql-ai-functions%2F05-Extra-setup-external-model-OpenAI&version=1">

## Store your key using Databricks secrets

We'll use [Databricks secrets](https://docs.databricks.com/security/secrets/index.html) to hold our API tokens. Use the [Databricks Secrets CLI](https://docs.databricks.com/dev-tools/cli/secrets-cli.html) or [Secrets API 2.0](https://docs.databricks.com/dev-tools/api/latest/secrets.html) to manage your secrets. The below examples use the Secrets CLI

- If you don't already have a secret scope to keep your OpenAI keys in, create one now: 
  `databricks secrets create-scope --scope dbdemos`
- You will need to give `READ` or higher access for principals (e.g. users, groups) who are allowed to connect to OpenAI. 
  - We recommend creating a group `openai-users` and adding permitted users to that group
  - Then give that group `READ` (or higher) permission to the scope: 
    `databricks secrets put-acl --scope dbdemos --principal openai-users --permission READ`
- Create a secret for your API access token. We recommend format `<resource-name>-key`: 
  `databricks secrets put --scope dbdemos --key azure-openai`

**Caution**: Do **NOT** include your token in plain text in your Notebook, code, or git repo

### Create an external model endpoint
To run the following code, connect this notebook to a <b> Standard interactive cluster </b>.

In [0]:
%pip install databricks-sdk==0.39.0 mlflow==2.9.0
dbutils.library.restartPython()

In [0]:
from mlflow.deployments import get_deploy_client
deploy_client = get_deploy_client("databricks")

try:
    endpoint_name  = "dbdemos-azure-openai"
    deploy_client.create_endpoint(
        name=endpoint_name,
        config={
            "served_entities": [
                {
                    "name": endpoint_name,
                    "external_model": {
                        "name": "gpt-35-turbo",
                        "provider": "openai", #can support openAI or other external models. see https://docs.databricks.com/en/generative-ai/external-models/index.html
                        "task": "llm/v1/chat",
                        "openai_config": {
                            "openai_api_type": "azure",
                            "openai_api_key": "{{secrets/dbdemos/azure-openai}}", #Replace with your own azure open ai key
                            "openai_deployment_name": "dbdemo-gpt35", #your deployment name
                            "openai_api_base": "https://dbdemos-open-ai.openai.azure.com/", #your api base
                            "openai_api_version": "2023-05-15"
                        }
                    }
                }
            ]
        }
    )
    print(f"Endpoint {endpoint_name} created successfully")
except Exception as e:
    if 'RESOURCE_ALREADY_EXISTS' in str(e):
        print('Endpoint already exists')
    else:
        raise e

## Your external model is now ready!

You can use it in your SQL AI function:

`SELECT AI_QUERY('dbdemos-azure-openai', 'This will be sent to azure openai!')``

Congrats! You're now ready to use any model within your SQL queries!

Go back to [the introduction]($./00-SQL-AI-Functions-Introduction)